In [1]:
# Import data from CSV files
from csv import DictReader

with open('_data/fluidized_bed.csv', 'r') as csv_file:
    reader = DictReader(csv_file)
    rows_fb = [r for r in reader]
csv_file.close()

with open('_data/batch_reactor.csv', 'r') as csv_file:
    reader = DictReader(csv_file)
    rows_br = [r for r in reader]
csv_file.close()

with open('_data/tubular_reactor.csv', 'r') as csv_file:
    reader = DictReader(csv_file)
    rows_tr = [r for r in reader]
csv_file.close()

INPUT_VARS = [
    'Reaction Time (s)',
    'Temperature (C)',
    'Particle size (mm)',
    'Moisture',
    'Ash',
    'C',
    'H',
    'N',
    'O'
]

TARGET_VARS = [
    'Oil',
    'Char',
    'Gas'
]

In [2]:
# Obtain spearman coefficients for every input/target relationship
from typing import List, Tuple
from scipy.stats import spearmanr

def get_inp_tar(rows: List[dict], inp: str, tar) -> Tuple[List[float]]:

    inputs = []
    targets = []
    for row in rows:
        try:
            _i = float(row[inp])
            _t = float(row[tar])
            inputs.append(_i)
            targets.append(_t)
        except:
            continue
    return (inputs, targets)

results = []
for inp in INPUT_VARS:
    for tar in TARGET_VARS:

        inputs_fb, targets_fb = get_inp_tar(rows_fb, inp, tar)
        inputs_br, targets_br = get_inp_tar(rows_br, inp, tar)
        inputs_tr, targets_tr = get_inp_tar(rows_tr, inp, tar)

        corr_fb, p_fb = spearmanr(inputs_fb, targets_fb)
        corr_br, p_br = spearmanr(inputs_br, targets_br)
        corr_tr, p_tr = spearmanr(inputs_tr, targets_tr)

        results.append({
            'Input Parameter': inp,
            'Yield': tar,
            'Rho (FB)': corr_fb,
            'P-value (FB)': p_fb,
            'Rho (BR)': corr_br,
            'P-value (BR)': p_br,
            'Rho (TR)': corr_tr,
            'P-value (TR)': p_tr
        })

In [3]:
# Dump spearman results to CSV file
from csv import DictWriter

headers = [
    'Input Parameter',
    'Yield',
    'Rho (FB)',
    'P-value (FB)',
    'Rho (BR)',
    'P-value (BR)',
    'Rho (TR)',
    'P-value (TR)'
]

with open('_results/spearman_results.csv', 'w', encoding='utf8') as csv_file:
    writer = DictWriter(csv_file, headers, delimiter=',', lineterminator='\n')
    writer.writeheader()
    writer.writerows(results)
csv_file.close()

In [4]:
# Highlight relationships with low (< 0.01) p-value (i.e., statistically significant)

for res in results:
    if res['P-value (FB)'] < 0.01:
        print(f"FB: {res['Input Parameter']} | {res['Yield']} | {res['Rho (FB)']} | {res['P-value (FB)']}")
    if res['P-value (BR)'] < 0.01:
        print(f"BR: {res['Input Parameter']} | {res['Yield']} | {res['Rho (BR)']} | {res['P-value (BR)']}")
    if res['P-value (TR)'] < 0.01:
        print(f"TR: {res['Input Parameter']} | {res['Yield']} | {res['Rho (TR)']} | {res['P-value (TR)']}")

BR: Reaction Time (s) | Oil | 0.6556051151898084 | 3.2819806132834324e-06
TR: Reaction Time (s) | Char | -0.8876253645985945 | 0.0006079255649158403
FB: Reaction Time (s) | Gas | -0.6356896058692534 | 0.001114604365707142
FB: Temperature (C) | Char | -0.8361011622140674 | 1.7107627110829508e-16
BR: Temperature (C) | Char | -0.886890197712825 | 1.1836893895275317e-14
TR: Temperature (C) | Char | -0.41186109437402824 | 0.0032786749877503855
FB: Temperature (C) | Gas | 0.736612839695015 | 2.933157960508126e-11
BR: Temperature (C) | Gas | 0.9179120518318602 | 3.0371928949728425e-17
TR: Temperature (C) | Gas | 0.6696868928330924 | 1.425861632832154e-07
BR: Particle size (mm) | Oil | 0.6936341488209178 | 0.0003436784066521996
TR: Particle size (mm) | Oil | -0.544007406776054 | 0.0010661971827252363
BR: Particle size (mm) | Char | 0.6200557271902728 | 0.002082180666578846
BR: Particle size (mm) | Gas | -0.7855333567237736 | 1.4810171268867073e-05
BR: Moisture | Char | -0.44882204382353585 | 0